In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split


In [3]:
data = pd.read_csv('data/German_data.csv')

In [4]:
data['Good/Bad'] = data['Good/Bad'].astype(int)
data['sex'] = LabelEncoder().fit_transform(data['sex']) 
# 区分数值型和分类型变量
numerical_features = data.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = data.select_dtypes(include=['object']).columns.tolist()
# print(categorical_features)
# 去掉good/bad 和sex列
numerical_features.remove('Good/Bad')
numerical_features.remove('sex')
#对分类变量进行独热编码
data_encoded = pd.get_dummies(data[categorical_features], drop_first=True)
data_encoded = data_encoded.astype(float)
data_preprocessed = pd.concat([data[numerical_features], data_encoded], axis=1)
# 分离特征和标签
X = data_preprocessed
y_label = data['Good/Bad']
y_sensitive = data['sex']

# 标准化特征
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 划分训练集和测试集
X_train, X_test, y_label_train, y_label_test, y_sensitive_train, y_sensitive_test = train_test_split(
    X_scaled, y_label, y_sensitive, test_size=0.2, random_state=42)

In [38]:
X_train[1]

array([ 2.24755338,  1.52682862, -0.86919627,  1.04808574, -0.39839302,
       -0.7039176 , -0.42854322,  1.64734776, -0.25943726, -0.80699422,
       -0.22710999, -1.06304467,  3.21749479, -0.64266066, -0.33905067,
        9.06917857, -0.47039493, -0.62249498, -0.11026357, -0.15005969,
       -0.22953657, -0.09534626, -0.32793109,  2.94941166, -0.25943726,
       -0.22466238, -0.47198116, -0.45604896, -0.71668428, -0.4592484 ,
       -0.58081787,  0.6707663 , -0.20687555, -0.23432935, -0.54997926,
       -0.70551479,  2.3424374 , -0.22219304,  0.47831191, -1.57506706,
        2.87228132, -0.50031279, -1.30384048,  2.39791576,  1.21610369,
       -0.19611614])

In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

2024-06-04 20:11:17.691857: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

# # 标签注释器
# input_dim = X_train.shape[1]
# label_input = Input(shape=(input_dim,), name='label_input')
# h = Dense(64, activation='relu')(label_input)
# h = Dense(32, activation='relu')(h)
# label_output = Dense(1, activation='sigmoid', name='label_output')(h)  # 这里添加(h)

# label_model = Model(inputs=label_input, outputs=label_output)
# label_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# label_model.fit(X_train, y_label_train, epochs=100, batch_size=16, validation_split=0.25)
# label_model.evaluate(X_test, y_label_test)


In [7]:
# 敏感属性注释器
# sensitive_input = Input(shape=(input_dim,), name='sensitive_input')
# h = Dense(64, activation='relu')(sensitive_input)
# h = Dense(32, activation='relu')(h)
# sensitive_output = Dense(1, activation='sigmoid', name='sensitive_output')(h)  # 这里添加(h)

# sensitive_model = Model(inputs=sensitive_input, outputs=sensitive_output)
# sensitive_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# sensitive_model.fit(X_train, y_sensitive_train, epochs=100, batch_size=32, validation_split=0.2)
# sensitive_model.evaluate(X_test, y_sensitive_test)


In [8]:
# import numpy as np
# from tensorflow.keras.layers import Input, Dense, LeakyReLU
# from tensorflow.keras.models import Model
# # # 生成器
# # generator_input = Input(shape=(input_dim,), name='generator_input')
# # h = Dense(128)(generator_input)
# # h = LeakyReLU(negative_slope=0.01)(h)  # Use LeakyReLU for better gradient flow
# # h = Dense(256)(h)
# # h = LeakyReLU(negative_slope=0.01)(h)
# # h = Dense(512)(h)
# # h = LeakyReLU(negative_slope=0.01)(h)
# # generator_output = Dense(input_dim, activation='linear', name='generator_output')(h)


# # generator_model = Model(inputs=generator_input, outputs=generator_output)
# # generator_model.compile(optimizer='adam', loss='mean_squared_error')  # 编译生成器模型

# # # 在训练循环中，训练生成器
# # noise = np.random.normal(0, 1, size=(len(X_train), input_dim))
# # generator_loss = generator_model.train_on_batch(noise, X_train)
# # print("Generator Loss:", generator_loss)

# # 生成器
# generator_input = Input(shape=(input_dim,), name='generator_input')
# h = Dense(128)(generator_input)
# h = LeakyReLU(negative_slope=0.01)(h)
# h = Dense(256)(h)
# h = LeakyReLU(negative_slope=0.01)(h)
# h = Dense(512)(h)
# h = LeakyReLU(negative_slope=0.01)(h)
# generator_output = Dense(input_dim, activation='linear', name='generator_output')(h)

# generator_model = Model(inputs=generator_input, outputs=generator_output)
# generator_model.compile(optimizer='adam', loss='mean_squared_error')  # 编译生成器模型
# # 在训练循环中，训练生成器
# noise = np.random.normal(0, 1, size=(len(X_train), input_dim))
# generator_loss = generator_model.train_on_batch(noise, X_train)
# print("Generator Loss:", generator_loss)


In [9]:
# # 分类器
# classifier_input = Input(shape=(input_dim,), name='classifier_input')
# h = Dense(128)(classifier_input)
# h = LeakyReLU(negative_slope=0.01)(h)
# h = Dense(256)(h)
# h = LeakyReLU(negative_slope=0.01)(h)
# h = Dense(512)(h)
# h = LeakyReLU(negative_slope=0.01)(h)
# h = Dense(256)(h)
# h = LeakyReLU(negative_slope=0.01)(h)
# h = Dense(128)(h)
# h = LeakyReLU(negative_slope=0.01)(h)
# classifier_output = Dense(1, activation='sigmoid', name='classifier_output')(h)

# classifier_model = Model(inputs=classifier_input, outputs=classifier_output)
# classifier_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# classifier_loss = classifier_model.train_on_batch(X_train, y_label_train)
# print(classifier_loss)

In [10]:
# import numpy as np

# # 定义交替训练的轮数和批次大小
# epochs = 100
# batch_size = 16

# # 交替训练生成器和分类器
# for epoch in range(epochs):
#     print("Epoch:", epoch + 1)
    
#     # 训练生成器
#     # 1. 使用标签注释器和敏感属性注释器生成标签和敏感属性的预测
#     label_predictions = label_model.predict(X_train)
#     sensitive_predictions = sensitive_model.predict(X_train)
    
#     # 2. 生成随机噪声
#     noise = np.random.normal(0, 1, size=(len(X_train), input_dim))
    
#     # 3. 使用生成器生成新的数据样本
#     generated_data = generator_model.predict(noise) 
    
#     # 4. 构建生成器训练数据，包括标签注释器和敏感属性注释器的预测结果
#     generator_input = np.concatenate([label_predictions, sensitive_predictions, noise], axis=1)
    
#     # 5. 训练生成器
#     generator_loss = generator_model.train_on_batch(noise, X_train)
#     # print("Generator Loss:", generator_loss)
    
#     # 训练分类器
#     # 1. 使用生成器生成的数据样本和真实数据样本构建训练数据
#     combined_data = np.concatenate([generated_data, X_train], axis=0)
#     combined_labels = np.concatenate([np.zeros(len(generated_data)), np.ones(len(X_train))])
    
#     # 2. 训练分类器
#     classifier_loss = classifier_model.train_on_batch(combined_data, combined_labels)
#     print("Generator Loss:", generator_loss)
#     print("Classifier Loss:", classifier_loss)
#     # 每一百次打印一次结果
#     # if (epoch + 1) % 100 == 0:
#         # print("Generator Loss:", generator_loss)
#         # print("Classifier Loss:", classifier_loss)


In [11]:
# # 使用生成器生成新的数据样本
# generated_samples = generator_model.predict(noise)


运行结果：注释器训练次数30，生成器/分类器训练次数1000，效果accuracy = 0
修改：尝试理解其中的结构：重新定义分类器

In [12]:
# # 使用分类器对生成的数据进行分类
# generated_labels = classifier_model.predict(generated_samples)
# print(generated_labels)
# # 计算分类器的性能指标
# # accuracy = classifier_model.evaluate(generated_samples, generated_labels)[1]
# # print("Classifier Accuracy on Generated Data:", accuracy)


In [13]:
# import numpy as np
# import matplotlib.pyplot as plt
# from tensorflow.keras.layers import Input, Dense, LeakyReLU
# from tensorflow.keras.models import Model
# from tensorflow.keras.optimizers import Adam

# # 更新生成器
# def build_generator(input_dim):
#     generator_input = Input(shape=(input_dim,), name='generator_input')
#     h = Dense(256)(generator_input)
#     h = LeakyReLU(negative_slope=0.01)(h)
#     h = Dense(512)(h)
#     h = LeakyReLU(negative_slope=0.01)(h)
#     h = Dense(1024)(h)
#     h = LeakyReLU(negative_slope=0.01)(h)
#     generator_output = Dense(input_dim, activation='linear', name='generator_output')(h)
#     generator_model = Model(inputs=generator_input, outputs=generator_output)
#     return generator_model

# # 更新分类器
# def build_classifier(input_dim):
#     classifier_input = Input(shape=(input_dim,), name='classifier_input')
#     h = Dense(128)(classifier_input)
#     h = LeakyReLU(negative_slope=0.01)(h)
#     h = Dense(256)(h)
#     h = LeakyReLU(negative_slope=0.01)(h)
#     h = Dense(512)(h)
#     h = LeakyReLU(negative_slope=0.01)(h)
#     h = Dense(256)(h)
#     h = LeakyReLU(negative_slope=0.01)(h)
#     h = Dense(128)(h)
#     h = LeakyReLU(negative_slope=0.01)(h)
#     classifier_output = Dense(1, activation='sigmoid', name='classifier_output')(h)
#     classifier_model = Model(inputs=classifier_input, outputs=classifier_output)
#     return classifier_model

# input_dim = X_train.shape[1]

# # 构建生成器和分类器
# generator_model = build_generator(input_dim)
# classifier_model = build_classifier(input_dim)

# # 编译分类器
# classifier_model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# # 构建联合模型：生成器 + 分类器
# classifier_model.trainable = False
# gan_input = Input(shape=(input_dim,))
# generated_data = generator_model(gan_input)
# gan_output = classifier_model(generated_data)
# gan_model = Model(gan_input, gan_output)
# gan_model.compile(optimizer=Adam(), loss='binary_crossentropy')

# # 定义交替训练的轮数和批次大小
# epochs = 1000
# batch_size = 32

# # 存储损失
# generator_losses = []
# classifier_losses = []

# # 交替训练生成器和分类器
# for epoch in range(epochs):
#     if (epoch + 1) % 100 == 0:
#         print("Epoch:", epoch + 1)
    
#     # 训练分类器
#     # 1. 生成真实数据和生成数据的标签
#     label_predictions = label_model.predict(X_train, verbose=0)
#     sensitive_predictions = sensitive_model.predict(X_train, verbose=0)
    
#     # 2. 生成随机噪声
#     noise = np.random.normal(0, 1, size=(len(X_train), input_dim))
    
#     # 3. 使用生成器生成新的数据样本
#     generated_data = generator_model.predict(noise, verbose=0)
    
#     # 4. 构建分类器训练数据
#     combined_data = np.concatenate([generated_data, X_train], axis=0)
#     combined_labels = np.concatenate([np.zeros(len(generated_data)), np.ones(len(X_train))])
    
#     # 5. 训练分类器
#     classifier_loss = classifier_model.train_on_batch(combined_data, combined_labels)
    
#     # 训练生成器
#     # 1. 生成随机噪声
#     noise = np.random.normal(0, 1, size=(batch_size, input_dim))
    
#     # 2. 训练生成器（使生成的数据被分类器认为是真实的）
#     generator_loss = gan_model.train_on_batch(noise, np.ones((batch_size, 1)))
    
#     # 存储损失
#     generator_losses.append(generator_loss)
#     classifier_losses.append(classifier_loss[0])
    
#     # 每一百次打印一次结果
#     if (epoch + 1) % 100 == 0:
#         print("Generator Loss:", generator_loss)
#         print("Classifier Loss:", classifier_loss)

# # 可视化训练过程中的损失
# plt.plot(generator_losses, label='Generator Loss')
# plt.plot(classifier_losses, label='Classifier Loss')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()

# # 生成样本数据
# generated_samples = generator_model.predict(np.random.normal(0, 1, size=(10, input_dim)))

# # 使用分类器对生成的数据进行分类
# generated_labels = classifier_model.predict(generated_samples)
# print(generated_labels)


In [14]:
# # 生成样本数据
# generated_samples = generator_model.predict(np.random.normal(0, 1, size=(10, input_dim)))
# print(generated_samples)
# print('here the original data')
# print(X_train)

# # # 使用分类器对生成的数据进行分类
# # generated_labels = classifier_model.predict(generated_samples)
# # print(generated_labels)

上述代码效果不好，分析原因：分类器只是进行了分类，并没有告诉生成器生成的数据是好是坏，因此不能够得到收敛的结果
但是如果不这样，直接还是使用判别器，我应当如何训练分类器？难道是GAN作为生成的一部分，然后每次循环当中再加上分类器的训练？

In [15]:
#分类器
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 训练分类器
classifier = RandomForestClassifier()
classifier.fit(X_train, y_label_train)

# 预测
y_label_pred = classifier.predict(X_test)

# 评估分类器性能
accuracy = accuracy_score(y_label_test, y_label_pred)
precision = precision_score(y_label_test, y_label_pred)
recall = recall_score(y_label_test, y_label_pred)
f1 = f1_score(y_label_test, y_label_pred)

print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1 Score: {f1}")


Accuracy: 0.705, Precision: 0.7294117647058823, Recall: 0.9051094890510949, F1 Score: 0.8078175895765473


In [40]:
import torch
import torch.nn as nn
import torch.optim as optim

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(100, 256),
            nn.ReLU(True),
            nn.Linear(256, 512),
            nn.ReLU(True),
            nn.Linear(512, 256),
            nn.ReLU(True),
            nn.Linear(256, X_train.shape[1]),
            nn.Tanh()
        )
    
    def forward(self, input):
        return self.main(input)

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(X_train.shape[1], 1024),
            nn.ReLU(True),
            nn.Linear(1024, 512),
            nn.ReLU(True),
            nn.Linear(512, 256),
            nn.ReLU(True),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
    
    def forward(self, input):
        return self.main(input)

netG = Generator()
netD = Discriminator()

criterion = nn.BCELoss()
optimizerD = optim.Adam(netD.parameters(), lr=0.0002)
optimizerG = optim.Adam(netG.parameters(), lr=0.0002)

# 训练GAN
for epoch in range(5000):
    # 更新判别器
    netD.zero_grad()
    real_data = torch.Tensor(X_train)
    batch_size = real_data.size(0)
    labels = torch.ones((batch_size, 1))  # 创建一个形状为(batch_size, 1)的张量
    output = netD(real_data)
    lossD_real = criterion(output, labels)
    lossD_real.backward()

    # noise = torch.randn(batch_size, 100)
    # 例如，将均匀分布和正态分布进行混合
    uniform_noise = torch.rand(batch_size, 50)  # 生成均匀分布的随机数
    normal_noise = torch.randn(batch_size, 50)  # 生成正态分布的随机数
    noise = torch.cat((uniform_noise, normal_noise), dim=1)  # 将两种噪声分布进行拼接

    fake_data = netG(noise)
    labels = torch.zeros(batch_size,1)
    output = netD(fake_data.detach())
    lossD_fake = criterion(output, labels)
    lossD_fake.backward()
    

    optimizerD.step()

    # 更新生成器
    netG.zero_grad()
    labels = torch.ones(batch_size,1)
    output = netD(fake_data)
    lossG = criterion(output, labels)
    lossG.backward()
    optimizerG.step()

    if epoch % 100 == 0:
        print(f'Epoch [{epoch}/5000], Loss D: {lossD_real + lossD_fake}, Loss G: {lossG}')


Epoch [0/5000], Loss D: 1.3731852769851685, Loss G: 0.683114767074585
Epoch [100/5000], Loss D: 0.021458474919199944, Loss G: 3.971863269805908
Epoch [200/5000], Loss D: 0.009374379180371761, Loss G: 8.97299575805664
Epoch [300/5000], Loss D: 0.00019671082554850727, Loss G: 11.472001075744629
Epoch [400/5000], Loss D: 4.0322055383512634e-07, Loss G: 15.104263305664062
Epoch [500/5000], Loss D: 2.9477760108420625e-05, Loss G: 10.691465377807617
Epoch [600/5000], Loss D: 1.374008661514381e-05, Loss G: 11.4887113571167
Epoch [700/5000], Loss D: 0.03237169235944748, Loss G: 18.481685638427734
Epoch [800/5000], Loss D: 1.4919816770930794e-10, Loss G: 37.462650299072266
Epoch [900/5000], Loss D: 1.4919816770930794e-10, Loss G: 37.46120071411133
Epoch [1000/5000], Loss D: 1.4919816770930794e-10, Loss G: 37.46110534667969
Epoch [1100/5000], Loss D: 1.4919816770930794e-10, Loss G: 37.46107482910156
Epoch [1200/5000], Loss D: 1.4919816770930794e-10, Loss G: 37.461063385009766
Epoch [1300/5000], 

In [39]:
# 生成数据
import numpy as np
noise = torch.randn(10, 100)
generated_data = netG(noise).detach().numpy()
print(generated_data)
print(X_train)
# 注释生成的数据
def label_annotator(data, classifier):
    return classifier.predict(data)

labels = label_annotator(generated_data, classifier)
print(labels)
# 敏感类型注释器
def sensitive_type_annotator(data):
    # 假设敏感信息在数据的特定列
    # 在这个例子中，假设最后一列是敏感信息（性别）
    sensitive_info = np.zeros(data.shape[0])  # 假设所有生成数据的敏感类型都为0
    return sensitive_info

sensitive_info = sensitive_type_annotator(generated_data)
# print(sensitive_info)

[[ 1.         -1.         -1.          1.          1.          1.
   1.          1.          1.          1.         -1.         -1.
  -1.         -1.          1.         -1.          1.         -1.
   1.         -1.         -1.         -1.         -1.          1.
   1.          1.          1.          1.         -1.          1.
  -1.         -1.         -1.         -1.         -1.          1.
  -1.          1.         -1.          1.         -1.          1.
   1.          1.         -1.          1.        ]
 [ 1.         -1.         -1.          1.          1.          1.
   1.          1.          1.          1.         -1.         -1.
  -1.         -1.          1.         -1.          1.         -1.
   1.         -1.         -1.         -1.         -1.          1.
   0.9999997   1.          1.          1.         -1.          0.9999999
  -1.         -1.         -1.         -1.         -1.          1.
  -1.          1.         -1.          0.99999994 -1.          1.
   1.          1. 

In [25]:
def evaluate_fairness(labels, sensitive_info):
    sensitive_groups = np.unique(sensitive_info)
    for group in sensitive_groups:
        group_labels = labels[sensitive_info == group]
        print(f'Group {group}, label distribution: {np.mean(group_labels)}')

evaluate_fairness(labels, sensitive_info)


Group 0.0, label distribution: 1.0


In [2]:
from train import train_fn
dir(train_fn)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'c_compute_loss',
 'c_train_step',
 'c_update',
 'compute_loss',
 'generated_population_stats',
 'get_mu_vars',
 'init_classifier',
 'init_gen_opt',
 'load',
 'predict',
 'reparation_batch',
 'sample_decoder',
 'save',
 'train_classifier',
 'train_generator',
 'train_step',
 'update',
 'validate',
 'visualize_outputs']